In [1]:
import pandas as pd
import numpy as np
import os, sys, glob
import matplotlib.pyplot as plt
import xarray as xr

In [16]:
# refe data
ds_summa = xr.load_dataset('/glade/campaign/cgd/tss/people/mozhgana/projects/SUMMA_Calib/summa_camels/01013500/simulations/summa_iter0_trial132/camels_era5l_day.nc')
ds_summa

<xarray.Dataset> Size: 133kB
Dimensions:                     (time: 2557, hru: 1, gru: 1)
Coordinates:
  * time                        (time) datetime64[ns] 20kB 1982-10-01 ... 198...
  * hru                         (hru) int64 8B 1013500
  * gru                         (gru) int64 8B 1013500
Data variables: (12/13)
    pptrate_mean                (time, hru) float32 10kB 0.0001211 ... 4.298e-07
    airtemp_mean                (time, hru) float32 10kB 286.4 282.5 ... 279.4
    averageRoutedRunoff_mean    (time, gru) float32 10kB 4.374e-06 ... 1.528e-08
    scalarCanopyWat             (time, hru) float32 10kB 0.7043 ... 0.008119
    scalarSWE                   (time, hru) float32 10kB 0.0 0.0 ... 1.282e-08
    scalarAquiferStorage        (time, hru) float32 10kB 0.01431 ... 0.003897
    ...                          ...
    scalarSurfaceRunoff_mean    (time, hru) float32 10kB 1.586e-13 ... 1.024e-20
    scalarAquiferBaseflow_mean  (time, hru) float32 10kB 1.552e-05 ... 1.42e-08
    scalarTotalET_mean          (time, hru) float32 10kB -9.392e-06 ... -9.62...
    scalarTotalRunoff_mean      (time, hru) float32 10kB 1.552e-05 ... 1.42e-08
    hruId                       (hru) int64 8B 1013500
    gruId                       (gru) int64 8B 1013500
Attributes: (12/34)
    summaVersion:     undefined
    buildTime:        Thu 25 Apr 2024 08:00:41 AM MDT
    gitBranch:        master-0-g8321632d
    gitHash:          8321632d84135f70a4d6d31e2caa34208b7b9c00
    soilCatTbl:       STAS
    vegeParTbl:       MODIFIED_IGBP_MODIS_NOAH
    ...               ...
    thCondSnow:       jrdn1991
    thCondSoil:       funcSoilWet
    spatial_gw:       localColumn
    subRouting:       timeDlay
    history:          Fri Aug 30 19:07:10 2024: ncap2 -O -s time[time]=time-8...
    NCO:              netCDF Operators version 5.2.4 (Homepage = http://nco.s...

In [17]:
# ref data
ds_topo = xr.load_dataset('ntopo_MERIT_Hydro_v1_01013500.nc')
ds_topo

<xarray.Dataset> Size: 3kB
Dimensions:     (seg: 51, hru: 51)
Dimensions without coordinates: seg, hru
Data variables:
    Length      (seg) float64 408B 1.094e+04 4.494e+03 ... 4.977e+03 8.653e+03
    seg_id      (seg) int32 204B 72040863 72040882 ... 72045140 72045221
    Tosegment   (seg) int32 204B -9999 72040863 72040882 ... 72042628 72042404
    Slope       (seg) float64 408B 0.001082 4.447e-05 ... 0.0006023 0.0002311
    pfaf        (seg) |S11 561B b'L0C52R7613' b'L0C52R7631' ... b'L0C52R7649'
    Basin_Area  (hru) float64 408B 3.789e+07 1.815e+07 ... 3.767e+07 8.745e+07
    hru_seg_id  (hru) int32 204B 72040863 72040882 ... 72045140 72045221
    hruid       (hru) int32 204B 72040863 72040882 ... 72045140 72045221
Attributes:
    history:  Wed Jul 17 09:42:06 2024: ./scripts/subset_river_network.py /gl...
    NCO:      netCDF Operators version 5.1.6 (Homepage = http://nco.sf.net, C...

In [2]:
basin = 0
flag = 'iter0_trial0'
year=2010
# file=f'/glade/work/guoqiang/CTSM_CAMELS/data_mesh_surf/HillslopeHydrology/disaggregation/surfdata_CAMELS_level1_hist_78pfts_CMIP6_simyr2000_HAND_trapezoidal_{basin}.nc'
# ds_surf = xr.load_dataset(file)
# mesharea = ds_surf.AREA.values

In [3]:
inpath = f'/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_emulator/level1_{basin}_calib/ctsm_outputs/{flag}/lnd/hist'
files = glob.glob(f'{inpath}/level1_*.clm2.h1.{year}-*.nc')
files.sort()
ds_clm = xr.open_mfdataset(files)

In [35]:
ds_clm_out = ds_clm[['QRUNOFF']].load()
ds_clm_out = ds_clm_out.rename({'lndgrid':'gru'})
ds_clm_out.coords['gru'] = [1013500]
ds_clm_out['gruId'] = xr.DataArray([1013500], dims=('gru'))
# ds_clm_out.coords['time'] = pd.date_range('2010-10-01', '2011-09-30')
# ds_clm_out.coords['time'] = ds_summa.time[0:365]
ds_clm_out

<xarray.Dataset> Size: 4kB
Dimensions:  (time: 365, gru: 1)
Coordinates:
  * time     (time) object 3kB 2010-10-01 00:00:00 ... 2011-09-30 00:00:00
  * gru      (gru) int64 8B 1013500
Data variables:
    QRUNOFF  (time, gru) float32 1kB 6.024e-05 0.0001588 ... 2.1e-06 3.646e-06
    gruId    (gru) int64 8B 1013500
Attributes: (12/101)
    title:                                CLM History file information
    comment:                              NOTE: None of the variables are wei...
    Conventions:                          CF-1.0
    history:                              created on 10/09/24 18:50:58
    source:                               Community Terrestrial Systems Model
    hostname:                             derecho
    ...                                   ...
    cft_irrigated_tropical_corn:          62
    cft_tropical_soybean:                 63
    cft_irrigated_tropical_soybean:       64
    time_period_freq:                     day_1
    Time_constant_3Dvars_filename:        ./level1_0_iter0_trial0.clm2.h0.200...
    Time_constant_3Dvars:                 ZSOI:DZSOI:WATSAT:SUCSAT:BSW:HKSAT:...

In [37]:
ds_clm

<xarray.Dataset> Size: 180kB
Dimensions:                        (levgrnd: 25, levsoi: 20, levlak: 10,
                                    levdcmp: 1, time: 365, hist_interval: 2,
                                    lndgrid: 1, levsno: 12)
Coordinates:
  * levgrnd                        (levgrnd) float32 100B 0.01 0.04 ... 42.0
  * levsoi                         (levsoi) float32 80B 0.01 0.04 ... 6.94 8.03
  * levlak                         (levlak) float32 40B 0.05 0.6 ... 34.33 44.78
  * levdcmp                        (levdcmp) float32 4B 1.0
  * time                           (time) object 3kB 2010-10-01 00:00:00 ... ...
Dimensions without coordinates: hist_interval, lndgrid, levsno
Data variables: (12/48)
    mcdate                         (time) int32 1kB dask.array<chunksize=(365,), meta=np.ndarray>
    mcsec                          (time) int32 1kB dask.array<chunksize=(365,), meta=np.ndarray>
    mdcur                          (time) int32 1kB dask.array<chunksize=(365,), meta=np.ndarray>
    mscur                          (time) int32 1kB dask.array<chunksize=(365,), meta=np.ndarray>
    nstep                          (time) int32 1kB dask.array<chunksize=(365,), meta=np.ndarray>
    time_bounds                    (time, hist_interval) object 6kB dask.array<chunksize=(365, 2), meta=np.ndarray>
    ...                             ...
    ZWT                            (time, lndgrid) float32 1kB dask.array<chunksize=(365, 1), meta=np.ndarray>
    SNO_MELT                       (time, levsno, lndgrid) float32 18kB dask.array<chunksize=(365, 12, 1), meta=np.ndarray>
    SNO_T                          (time, levsno, lndgrid) float32 18kB dask.array<chunksize=(365, 12, 1), meta=np.ndarray>
    SNO_Z                          (time, levsno, lndgrid) float32 18kB dask.array<chunksize=(365, 12, 1), meta=np.ndarray>
    SOILICE                        (time, levsoi, lndgrid) float32 29kB dask.array<chunksize=(365, 20, 1), meta=np.ndarray>
    SOILLIQ                        (time, levsoi, lndgrid) float32 29kB dask.array<chunksize=(365, 20, 1), meta=np.ndarray>
Attributes: (12/101)
    title:                                CLM History file information
    comment:                              NOTE: None of the variables are wei...
    Conventions:                          CF-1.0
    history:                              created on 10/09/24 18:50:58
    source:                               Community Terrestrial Systems Model
    hostname:                             derecho
    ...                                   ...
    cft_irrigated_tropical_corn:          62
    cft_tropical_soybean:                 63
    cft_irrigated_tropical_soybean:       64
    time_period_freq:                     day_1
    Time_constant_3Dvars_filename:        ./level1_0_iter0_trial0.clm2.h0.200...
    Time_constant_3Dvars:                 ZSOI:DZSOI:WATSAT:SUCSAT:BSW:HKSAT:...

In [38]:
xr.open_dataset('/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_emulator/level1_0_calib/ctsm_outputs/iter0_trial0/lnd/hist/level1_0_iter0_trial0.clm2.h1.2008-10-01-00000.nc')

<xarray.Dataset> Size: 180kB
Dimensions:                        (levgrnd: 25, levsoi: 20, levlak: 10,
                                    levdcmp: 1, time: 365, hist_interval: 2,
                                    lndgrid: 1, levsno: 12)
Coordinates:
  * levgrnd                        (levgrnd) float32 100B 0.01 0.04 ... 42.0
  * levsoi                         (levsoi) float32 80B 0.01 0.04 ... 6.94 8.03
  * levlak                         (levlak) float32 40B 0.05 0.6 ... 34.33 44.78
  * levdcmp                        (levdcmp) float32 4B 1.0
  * time                           (time) object 3kB 2008-10-01 00:00:00 ... ...
Dimensions without coordinates: hist_interval, lndgrid, levsno
Data variables: (12/48)
    mcdate                         (time) int32 1kB ...
    mcsec                          (time) int32 1kB ...
    mdcur                          (time) int32 1kB ...
    mscur                          (time) int32 1kB ...
    nstep                          (time) int32 1kB ...
    time_bounds                    (time, hist_interval) object 6kB ...
    ...                             ...
    ZWT                            (time, lndgrid) float32 1kB ...
    SNO_MELT                       (time, levsno, lndgrid) float32 18kB ...
    SNO_T                          (time, levsno, lndgrid) float32 18kB ...
    SNO_Z                          (time, levsno, lndgrid) float32 18kB ...
    SOILICE                        (time, levsoi, lndgrid) float32 29kB ...
    SOILLIQ                        (time, levsoi, lndgrid) float32 29kB ...
Attributes: (12/99)
    title:                                CLM History file information
    comment:                              NOTE: None of the variables are wei...
    Conventions:                          CF-1.0
    history:                              created on 10/09/24 18:47:54
    source:                               Community Terrestrial Systems Model
    hostname:                             derecho
    ...                                   ...
    cft_irrigated_switchgrass:            60
    cft_tropical_corn:                    61
    cft_irrigated_tropical_corn:          62
    cft_tropical_soybean:                 63
    cft_irrigated_tropical_soybean:       64
    time_period_freq:                     day_1